This notebook is aiming at iterating active learning process after receving feedback from SME.

Prerequisite:

1) SME Input Label for testing data

2) Original Test data including filename, page number and text content

3) Existing training data

Output:

1) A csv file containing prepared training data ready for Step4

##### Part I: Reformat SME Input Label

In [1]:
# Import required libraries
import pandas as pd
import numpy as np

In [2]:
# Specify label .xlsx filename and location
key_file_addr = "Grading Details - 12Dec2022.xlsx"

In [3]:
# Load label file
key_file = pd.read_excel(key_file_addr)
key_file

BatchName  CR_Id  \
0                 Multi-Step-Non-OCR data – UAT Batch 1   5967   
1                 Multi-Step-Non-OCR data – UAT Batch 1   5967   
2                 Multi-Step-Non-OCR data – UAT Batch 1   5967   
3                 Multi-Step-Non-OCR data – UAT Batch 1   5967   
4                 Multi-Step-Non-OCR data – UAT Batch 1   5967   
...                                                 ...    ...   
1449  Multi-Step-Non-OCR data – UAT Batch 1 Iteration-2   6053   
1450  Multi-Step-Non-OCR data – UAT Batch 1 Iteration-2   6038   
1451  Multi-Step-Non-OCR data – UAT Batch 1 Iteration-2   6040   
1452  Multi-Step-Non-OCR data – UAT Batch 1 Iteration-2   6040   
1453  Multi-Step-Non-OCR data – UAT Batch 1 Iteration-2   6040   

                            CRFileName  \
0     1003700000000865685_11514934.pdf   
1     1003700000000865685_11514934.pdf   
2     1003700000000865685_11514934.pdf   
3     1003700000000865685_11514934.pdf   
4     1003700000000865685_11514934.pdf   
...                                ...   
1449  1003700000000849387_11513862.pdf   
1450  1003700000000849140_11520035.pdf   
1451  1003700000000849181_11515304.pdf   
1452  1003700000000849181_11515304.pdf   
1453  1003700000000849181_11515304.pdf   

                              User entered Pages  \
0                                           4,48   
1                                             48   
2                                          94-96   
3                                          88-93   
4                                           48,4   
...                                          ...   
1449  57,58,59, 60,61, 62,63,64,65,66,67, 68, 69   
1450                                     109,110   
1451                                     194,195   
1452                                         NaN   
1453                                         NaN   

                                    Keywords   SubsectionKeyword  \
0                                        NaN                 NaN   
1                                        NaN  Updates, Diagnosis   
2                          Discharge Summary                 NaN   
3                                  H&P Notes                 NaN   
4                                        NaN  Admit to Inpatient   
...                                      ...                 ...   
1449                                     NaN                 NaN   
1450  Care Manager Direct Patient Evaluation                 NaN   
1451                                     NaN                 NaN   
1452                                     NaN                 NaN   
1453                                     NaN                 NaN   

     MedicalKeyword            CRFieldName  \
0               NaN             admit_type   
1               NaN       admission_reason   
2               NaN      discharge_summary   
3               NaN       history_physical   
4               NaN        admission_order   
...             ...                    ...   
1449            NaN                consult   
1450            NaN        case_management   
1451            NaN  discharge_disposition   
1452            NaN        physician_query   
1453            NaN             flowsheets   

                                            UserComment  Accuracy  \
0                                                   NaN     100.0   
1     AR- Missed to capture page with keywords 'Upda...       0.0   
2     DS- Missed to capture page with keyword 'Disch...       0.0   
3     H&P- Missed to capture page with keyword 'H&P ...       0.0   
4     AO- Captured related but not appropriate page ...      45.0   
...                                                 ...       ...   
1449  Missed to extract first page in page range and...      85.0   
1450  CM- Missed to capture keyword Section: 'Care M...       0.0   
1451                                                NaN       0.0   
1452                                                NaN     100

In [4]:
# Check how many labels are in label file 
label_from_all = key_file['CRFieldName']
label_from_all.unique()

array(['admit_type', 'admission_reason', 'discharge_summary',
       'history_physical', 'admission_order', 'ed', 'consult',
       'radiology', 'labs', 'flowsheets', 'mar', 'case_management',
       'progress_notes', 'discharge_disposition', 'nutrition_assessment',
       'physician_query', 'procedures'], dtype=object)

In [5]:
# Replace the class with ones identical to current training data
key_file = key_file.replace({'CRFieldName' : {'ed' : 'emergency_department', 'nutrition_assessment' : 'nutritional_assessment'}})

In [6]:
# Drop empty labels records
valid_label_df = key_file.dropna(subset = ['User entered Pages'])
valid_label_df

BatchName  CR_Id  \
0                 Multi-Step-Non-OCR data – UAT Batch 1   5967   
1                 Multi-Step-Non-OCR data – UAT Batch 1   5967   
2                 Multi-Step-Non-OCR data – UAT Batch 1   5967   
3                 Multi-Step-Non-OCR data – UAT Batch 1   5967   
4                 Multi-Step-Non-OCR data – UAT Batch 1   5967   
...                                                 ...    ...   
1447  Multi-Step-Non-OCR data – UAT Batch 1 Iteration-2   6038   
1448  Multi-Step-Non-OCR data – UAT Batch 1 Iteration-2   6038   
1449  Multi-Step-Non-OCR data – UAT Batch 1 Iteration-2   6053   
1450  Multi-Step-Non-OCR data – UAT Batch 1 Iteration-2   6038   
1451  Multi-Step-Non-OCR data – UAT Batch 1 Iteration-2   6040   

                            CRFileName  \
0     1003700000000865685_11514934.pdf   
1     1003700000000865685_11514934.pdf   
2     1003700000000865685_11514934.pdf   
3     1003700000000865685_11514934.pdf   
4     1003700000000865685_11514934.pdf   
...                                ...   
1447  1003700000000849140_11520035.pdf   
1448  1003700000000849140_11520035.pdf   
1449  1003700000000849387_11513862.pdf   
1450  1003700000000849140_11520035.pdf   
1451  1003700000000849181_11515304.pdf   

                              User entered Pages  \
0                                           4,48   
1                                             48   
2                                          94-96   
3                                          88-93   
4                                           48,4   
...                                          ...   
1447                                         281   
1448                                          68   
1449  57,58,59, 60,61, 62,63,64,65,66,67, 68, 69   
1450                                     109,110   
1451                                     194,195   

                                    Keywords   SubsectionKeyword  \
0                                        NaN                 NaN   
1                                        NaN  Updates, Diagnosis   
2                          Discharge Summary                 NaN   
3                                  H&P Notes                 NaN   
4                                        NaN  Admit to Inpatient   
...                                      ...                 ...   
1447                                     NaN                 NaN   
1448                                     NaN                 NaN   
1449                                     NaN                 NaN   
1450  Care Manager Direct Patient Evaluation                 NaN   
1451                                     NaN                 NaN   

     MedicalKeyword            CRFieldName  \
0               NaN             admit_type   
1               NaN       admission_reason   
2               NaN      discharge_summary   
3               NaN       history_physical   
4               NaN        admission_order   
...             ...                    ...   
1447            NaN        admission_order   
1448            NaN       admission_reason   
1449            NaN                consult   
1450            NaN        case_management   
1451            NaN  discharge_disposition   

                                            UserComment  Accuracy  \
0                                                   NaN     100.0   
1     AR- Missed to capture page with keywords 'Upda...       0.0   
2     DS- Missed to capture page with keyword 'Disch...       0.0   
3     H&P- Missed to capture page with keyword 'H&P ...       0.0   
4     AO- Captured related but not appropriate page ...      45.0   
...                                                 ...       ...   
1447                                                NaN      90.0   
1448                                                NaN      90.0   
1449  Missed to extract first page in page range and...      85.0   
1450  CM- Missed to capture keyword Section: 'Care M...       0

In [7]:
# Organize training data
df = valid_label_df

In [8]:
modified_df =  pd.DataFrame()

for i in range(len(df)):
    col = 'User entered Pages'
    filename = df.iloc[i]['CRFileName'].split("_")[-2] + "_" + df.iloc[i]['CRFileName'].split("_")[-1]
    filename_cut = filename.split('_')[-2][-6:]
    print(filename)
    ds =  df.iloc[i][col]
    temp_list = []
    if str(ds).startswith('-'):
        ds = str(ds)[1:]
    if str(ds).endswith('-') :
        ds = str(ds)[:-1]

    try:
        if not pd.isna(df.iloc[i][col]):
            try:
                ds = int(ds)
                # Remove labels with page 0
                if ds == 0:
                    continue
                else:
                    temp_list.append(ds)
            except:
                if (ds.split(',').__len__() > 1):
                    for j in range(len(ds.split(','))):
                        if '-' in ds.split(',')[j].strip():
                            min_no = int(ds.split(',')[j].split('-')[0])
                            max_no = int(ds.split(',')[j].split('-')[1])
                            temp_list.extend(list(range(min_no, max_no + 1)))
                        elif '-' not in ds.split(',')[j].strip():
                            if ds.split(',')[j].strip().split('-') == ['']:
                                pass
                            else:
                                min_no = int(ds.split(',')[j].strip().split('-')[0])
                                temp_list.extend([min_no])
                else:
                    try:
                        for j in range(len(ds.split(','))):
                            min_no = int(ds.split(',')[j].split('-')[0])
                            max_no = int(ds.split(',')[j].split('-')[1])
                            temp_list.extend(list(range(min_no,max_no + 1)))

                        if len(ds.split(',')[j].split('-')) == 1:
                            temp_list.append(int(ds))

                    except Exception as e:
                        print(e)

        modified_df = pd.concat([modified_df, pd.DataFrame.from_dict ({'page':temp_list,'class': df.iloc[i]['CRFieldName'] , 'filename' : filename, 'filename_cut' : filename_cut})], ignore_index = True)
    except Exception as e:
        print(e)
        
# Convert Page to Int
modified_df['page'] = modified_df['page'].astype(int)

1003700000000865685_11514934.pdf
1003700000000865685_11514934.pdf
1003700000000865685_11514934.pdf
1003700000000865685_11514934.pdf
1003700000000865685_11514934.pdf
1003700000000865685_11514934.pdf
1003700000000865685_11514934.pdf
1003700000000865685_11514934.pdf
1003700000000865685_11514934.pdf
1003700000000865685_11514934.pdf
1003700000000865685_11514934.pdf
1003700000000865685_11514934.pdf
1003700000000865685_11514934.pdf
1003700000000865685_11514934.pdf
1003700000000864567_11525529.pdf
1003700000000864567_11525529.pdf
1003700000000864567_11525529.pdf
1003700000000864823_11532041.pdf
1003700000000863823_11516334.pdf
1003700000000863823_11516334.pdf
1003700000000863823_11516334.pdf
1003700000000863823_11516334.pdf
1003700000000863823_11516334.pdf
1003700000000863823_11516334.pdf
1003700000000863823_11516334.pdf
1003700000000863823_11516334.pdf
1003700000000863823_11516334.pdf
1003700000000863823_11516334.pdf
1003700000000864801_11517933.pdf
1003700000000864801_11517933.pdf
1003700000

1003700000000848935_11521021.pdf
1003700000000848935_11521021.pdf
1003700000000849129_11514857.pdf
1003700000000770848_11515165.pdf
1003700000000770848_11515165.pdf
1003700000000770848_11515165.pdf
1003700000000848625_11525280.pdf
1003700000000765452_11513716.pdf
1003700000000848625_11525280.pdf
1003700000000765452_11513716.pdf
1003700000000848625_11525280.pdf
1003700000000848625_11525280.pdf
1003700000000765452_11513716.pdf
1003700000000765452_11513716.pdf
1003700000000765452_11513716.pdf
1003700000000848625_11525280.pdf
1003700000000848625_11525280.pdf
1003700000000848625_11525280.pdf
1003700000000765452_11513716.pdf
1003700000000765452_11513716.pdf
1003700000000848935_11521021.pdf
1003700000000771562_11519952.pdf
1003700000000771562_11519952.pdf
1003700000000771562_11519952.pdf
1003700000000849129_11514857.pdf
1003700000000849129_11514857.pdf
1003700000000849129_11514857.pdf
1003700000000849129_11514857.pdf
1003700000000849129_11514857.pdf
1003700000000770848_11515165.pdf
1003700000

1003700000000794590_11514798.pdf
1003700000000794590_11514798.pdf
1003700000000794590_11514798.pdf
1003700000000794590_11514798.pdf
1003700000000794590_11514798.pdf
1003700000000794590_11514798.pdf
1003700000000794590_11514798.pdf
1003700000000801159_11521823.pdf
1003700000000801159_11521823.pdf
1003700000000801159_11521823.pdf
1003700000000801159_11521823.pdf
1003700000000801159_11521823.pdf
1003700000000801159_11521823.pdf
1003700000000684295_11526093.pdf
1003700000000684295_11526093.pdf
1003700000000684295_11526093.pdf
1003700000000684295_11526093.pdf
1003700000000684295_11526093.pdf
1003700000000684295_11526093.pdf
1003700000000684295_11526093.pdf
1003700000000684295_11526093.pdf
1003700000000684295_11526093.pdf
1003700000000684295_11526093.pdf
1003700000000810804_11525147.pdf
1003700000000810804_11525147.pdf
1003700000000810804_11525147.pdf
1003700000000810804_11525147.pdf
1003700000000810804_11525147.pdf
1003700000000810804_11525147.pdf
1003700000000810804_11525147.pdf
1003700000

1003700000000837094_11525229.pdf
1003700000000837094_11525229.pdf
1003700000000837094_11525229.pdf
1003700000000837094_11525229.pdf
1003700000000837094_11525229.pdf
1003700000000835805_11515264.pdf
1003700000000850950_11520039.pdf
1003700000000864016_11514645.pdf
1003700000000801232_11517378.pdf
1003700000000801232_11517378.pdf
1003700000000801232_11517378.pdf
1003700000000801232_11517378.pdf
1003700000000801232_11517378.pdf
1003700000000801232_11517378.pdf
1003700000000801232_11517378.pdf
1003700000000837765_11515272.pdf
1003700000000838451_11514841.pdf
1003700000000838451_11514841.pdf
1003700000000838451_11514841.pdf
1003700000000838451_11514841.pdf
1003700000000838451_11514841.pdf
1003700000000838451_11514841.pdf
1003700000000838451_11514841.pdf
1003700000000838451_11514841.pdf
1003700000000838451_11514841.pdf
1003700000000849054_11516616.pdf
1003700000000849054_11516616.pdf
1003700000000849054_11516616.pdf
1003700000000849250_11528292.pdf
1003700000000849250_11528292.pdf
1003700000

In [9]:
# Save generated label file
modified_df.to_csv("nonocr_1_label.csv", index=False)
modified_df

page                  class                          filename  \
0         4             admit_type  1003700000000865685_11514934.pdf   
1        48             admit_type  1003700000000865685_11514934.pdf   
2        48       admission_reason  1003700000000865685_11514934.pdf   
3        94      discharge_summary  1003700000000865685_11514934.pdf   
4        95      discharge_summary  1003700000000865685_11514934.pdf   
...     ...                    ...                               ...   
15513    69                consult  1003700000000849387_11513862.pdf   
15514   109        case_management  1003700000000849140_11520035.pdf   
15515   110        case_management  1003700000000849140_11520035.pdf   
15516   194  discharge_disposition  1003700000000849181_11515304.pdf   
15517   195  discharge_disposition  1003700000000849181_11515304.pdf   

      filename_cut  
0           865685  
1           865685  
2           865685  
3           865685  
4           865685  
...            ...  
15513       849387  
15514       849140  
15515       849140  
15516       849181  
15517       849181  

[15518 rows x 4 columns]

In [10]:
# Cneck distribution of labels in page level
real_label = modified_df['class']
real_label.value_counts()

progress_notes            2899
flowsheets                2627
labs                      2099
mar                       1944
radiology                 1089
emergency_department       799
admit_type                 728
discharge_summary          692
consult                    659
history_physical           621
admission_order            376
case_management            266
procedures                 246
admission_reason           210
discharge_disposition      163
nutritional_assessment      57
physician_query             43
Name: class, dtype: int64

##### Part II: Merge Label with Text and Concatenate to Existing Training Data

In [2]:
# Load prior prepared extracted text files
textract_df = pd.read_csv("Textract_output_train05.csv")
textract_df.rename(columns={'file_name' : 'filename'}, inplace=True)
textract_df

filename  page  \
0       1003700000000684295_11526093.pdf     1   
1       1003700000000684295_11526093.pdf     2   
2       1003700000000684295_11526093.pdf     3   
3       1003700000000684295_11526093.pdf     4   
4       1003700000000684295_11526093.pdf     5   
...                                  ...   ...   
166159  1003700000000865685_11514934.pdf   298   
166160  1003700000000865685_11514934.pdf   299   
166161  1003700000000865685_11514934.pdf   300   
166162  1003700000000865685_11514934.pdf   301   
166163  1003700000000865685_11514934.pdf   302   

                                                  content  
0       P.O. Box 14465Lexington KY 40512RECEIVEDJune 2...  
1       HMRM25520659Member/Patient BRENDA K COOPERServ...  
2       HMRM25549771Member/Patient JEFFERY A WINDLESer...  
3       HMRM25512775Member/Patient JANE A WHITEService...  
4       HMRM25512777Member/Patient SANDRA L COXService...  
...                                                   ...  
166159  Cleveland ClinicCCF WESTON HOSPITALPTNT: Raven...  
166160  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raver ...  
166161  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raven ...  
166162  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raven ...  
166163  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raven ...  

[166164 rows x 3 columns]

In [3]:
# Change filename to same format 
textract_df['filename'] = textract_df['filename'].str.split('_').str[-2] + '_' + textract_df['filename'].str.split('_').str[-1]

In [4]:
# Load prior prepared reformat label file
label_df = pd.read_csv("nonocr_1_label.csv")

In [5]:
# Check common file
common_files = set(textract_df.filename).intersection(label_df.filename)
len(common_files)

77

In [6]:
# filter out uncommon file
textract_df = textract_df[textract_df.filename.isin(common_files)]
textract_df

filename  page  \
0       1003700000000684295_11526093.pdf     1   
1       1003700000000684295_11526093.pdf     2   
2       1003700000000684295_11526093.pdf     3   
3       1003700000000684295_11526093.pdf     4   
4       1003700000000684295_11526093.pdf     5   
...                                  ...   ...   
166159  1003700000000865685_11514934.pdf   298   
166160  1003700000000865685_11514934.pdf   299   
166161  1003700000000865685_11514934.pdf   300   
166162  1003700000000865685_11514934.pdf   301   
166163  1003700000000865685_11514934.pdf   302   

                                                  content  
0       P.O. Box 14465Lexington KY 40512RECEIVEDJune 2...  
1       HMRM25520659Member/Patient BRENDA K COOPERServ...  
2       HMRM25549771Member/Patient JEFFERY A WINDLESer...  
3       HMRM25512775Member/Patient JANE A WHITEService...  
4       HMRM25512777Member/Patient SANDRA L COXService...  
...                                                   ...  
166159  Cleveland ClinicCCF WESTON HOSPITALPTNT: Raven...  
166160  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raver ...  
166161  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raven ...  
166162  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raven ...  
166163  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raven ...  

[29483 rows x 3 columns]

In [7]:
# merge text files with labels
merged_file = pd.merge(textract_df, label_df, how = "left", on = ["filename", "page"])
merged_file

filename  page  \
0      1003700000000684295_11526093.pdf     1   
1      1003700000000684295_11526093.pdf     2   
2      1003700000000684295_11526093.pdf     3   
3      1003700000000684295_11526093.pdf     4   
4      1003700000000684295_11526093.pdf     5   
...                                 ...   ...   
32786  1003700000000865685_11514934.pdf   300   
32787  1003700000000865685_11514934.pdf   301   
32788  1003700000000865685_11514934.pdf   301   
32789  1003700000000865685_11514934.pdf   302   
32790  1003700000000865685_11514934.pdf   302   

                                                 content             class  \
0      P.O. Box 14465Lexington KY 40512RECEIVEDJune 2...               NaN   
1      HMRM25520659Member/Patient BRENDA K COOPERServ...               NaN   
2      HMRM25549771Member/Patient JEFFERY A WINDLESer...  history_physical   
3      HMRM25512775Member/Patient JANE A WHITEService...               NaN   
4      HMRM25512777Member/Patient SANDRA L COXService...               NaN   
...                                                  ...               ...   
32786  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raven ...    progress_notes   
32787  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raven ...    progress_notes   
32788  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raven ...    progress_notes   
32789  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raven ...    progress_notes   
32790  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raven ...    progress_notes   

       filename_cut  
0               NaN  
1               NaN  
2          684295.0  
3               NaN  
4               NaN  
...             ...  
32786      865685.0  
32787      865685.0  
32788      865685.0  
32789      865685.0  
32790      865685.0  

[32791 rows x 5 columns]

In [8]:
# Fill unlabelled data to "others" class
merged_file['class'] = merged_file['class'].fillna('others')
merged_file

filename  page  \
0      1003700000000684295_11526093.pdf     1   
1      1003700000000684295_11526093.pdf     2   
2      1003700000000684295_11526093.pdf     3   
3      1003700000000684295_11526093.pdf     4   
4      1003700000000684295_11526093.pdf     5   
...                                 ...   ...   
32786  1003700000000865685_11514934.pdf   300   
32787  1003700000000865685_11514934.pdf   301   
32788  1003700000000865685_11514934.pdf   301   
32789  1003700000000865685_11514934.pdf   302   
32790  1003700000000865685_11514934.pdf   302   

                                                 content             class  \
0      P.O. Box 14465Lexington KY 40512RECEIVEDJune 2...            others   
1      HMRM25520659Member/Patient BRENDA K COOPERServ...            others   
2      HMRM25549771Member/Patient JEFFERY A WINDLESer...  history_physical   
3      HMRM25512775Member/Patient JANE A WHITEService...            others   
4      HMRM25512777Member/Patient SANDRA L COXService...            others   
...                                                  ...               ...   
32786  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raven ...    progress_notes   
32787  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raven ...    progress_notes   
32788  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raven ...    progress_notes   
32789  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raven ...    progress_notes   
32790  Cleveland ClinicCCF WESTON HOSPITALPTNT:Raven ...    progress_notes   

       filename_cut  
0               NaN  
1               NaN  
2          684295.0  
3               NaN  
4               NaN  
...             ...  
32786      865685.0  
32787      865685.0  
32788      865685.0  
32789      865685.0  
32790      865685.0  

[32791 rows x 5 columns]

In [9]:
# Prepare data ready for "multi-label" mode
merged_file = merged_file[['content', 'class']].drop_duplicates().groupby('content')['class'].apply('|'.join).reset_index()

In [10]:
# Check the combined file
merged_file

content              class
0      "X-out" Pertinent NegativesAppropriate BoxesSu...             others
1      #6936206730026 (CD)Initic Post-Rov edica RACE ...             others
2      #6936471910986Incomplete Post-Pay Medal.Roold ...             others
3      #:DO00917407Date 09/27/21Acct#:DO0092275333Wei...   history_physical
4      %%STARTHRMCNINursing IntraopH21276001753906541...             others
...                                                  ...                ...
26039  y 08/29 0801 CMP EPON08/29 0801 DECADRON, HEXA...             others
26040  your spouses's GHP Information:Name and addres...             others
26041  your spouses's GHP Information:Name and addres...             others
26042  © 2000-2017 The StayWell Company, LLC. 780 Tow...  discharge_summary
26043  ©MedStar JLifeCareCareFliteTexasAMROtherAMB HE...             others

[26044 rows x 2 columns]

In [11]:
# To prepare for training data, only select "class" and "content" columns and keey it in right sequence
merged_file = merged_file[['class', 'content']]

In [12]:
# Save prepared training data to local
merged_file.to_csv("non-ocr-1-training-data.csv", index=False)

Load existing training data and combined with added training records

In [13]:
# Load prior training data
exist_training_df = pd.read_csv('batch3_combined_multi_label_additional.csv')
exist_training_df = exist_training_df[['class', 'content']]
exist_training_df

class  \
0       admission_reason|labs|admission_order|discharg...   
1                                       discharge_summary   
2                                                  others   
3                                                  others   
4                                                  others   
...                                                   ...   
832117                                             others   
832118                                             others   
832119                                             others   
832120                                             others   
832121                                             others   

                                                  content  
0       !! UNABLE TO FIND 4 tablets every morning. Med...  
1       !! predniSONE 20 MG tabletTake 40mg of prednis...  
2       !06$69608UPS Restrictions00000200 NOSLS LISIA ...  
3       !1Authorization is hereby given to dispense th...  
4       !I, AVILA, ANIBAL, and/or designee, have recei...  
...                                                   ...  
832117  © 2000-2021 The StayWell Company, LLC. All rig...  
832118  © 2016, Elsevier/Gold Standard. (2015-11-18 15...  
832119  àPage: 173PIERCE, HENRY CHARLESFac: St. Joseph...  
832120  àPage: 323PIERCE, HENRY CHARLESFac: St. Joseph...  
832121  ⑇~1356072~51260774~I&O_RPT~##ENHMC VRF ReportL...  

[832122 rows x 2 columns]

In [14]:
# Merge datasets together
merged_df = pd.concat([exist_training_df, merged_file], axis=0, ignore_index=True)
merged_df

class  \
0       admission_reason|labs|admission_order|discharg...   
1                                       discharge_summary   
2                                                  others   
3                                                  others   
4                                                  others   
...                                                   ...   
858161                                             others   
858162                                             others   
858163                                             others   
858164                                  discharge_summary   
858165                                             others   

                                                  content  
0       !! UNABLE TO FIND 4 tablets every morning. Med...  
1       !! predniSONE 20 MG tabletTake 40mg of prednis...  
2       !06$69608UPS Restrictions00000200 NOSLS LISIA ...  
3       !1Authorization is hereby given to dispense th...  
4       !I, AVILA, ANIBAL, and/or designee, have recei...  
...                                                   ...  
858161  y 08/29 0801 CMP EPON08/29 0801 DECADRON, HEXA...  
858162  your spouses's GHP Information:Name and addres...  
858163  your spouses's GHP Information:Name and addres...  
858164  © 2000-2017 The StayWell Company, LLC. 780 Tow...  
858165  ©MedStar JLifeCareCareFliteTexasAMROtherAMB HE...  

[858166 rows x 2 columns]

###### Attention: 

The dataframe loading in following code block (in this example "active_training_Dec142022.csv") should not contain keyword dataframe, as for downsample strategy we shouldn't dowmsample keyword dataframe. We should add the most recent keyword dataframe after the training dataframe is downsampled (balanced.)

In [15]:
# Save prepared training data locally
merged_df.to_csv('active_training_Dec142022.csv', index=False)

After this step, we can initiate downsample or multistep processing.

First we can re-direct to Analysis-1 Notebook to check the label distribution of current training data, then use that information to decide the number of records we need to downsample for each class.